# Необходимо написать скрипт для парсинга диалогов из файла test_data.csv. 

## Главные задачи, которые должен выполнять скрипт:
* Извлекать реплики с приветствием – где менеджер поздоровался. 
* Извлекать реплики, где менеджер представил себя. 
* Извлекать имя менеджера. 
* Извлекать название компании. 
* Извлекать реплики, где менеджер попрощался.
* Проверять требование к менеджеру: «В каждом диалоге обязательно необходимо поздороваться и попрощаться с клиентом»


In [188]:
# импортируем необходимые библиотеки
import re

import pandas as pd
import numpy as np

In [189]:
data = pd.read_csv('test_data.csv')
data.head()

,dlg_id,line_n,role,text
0,0,0,client,Алло
1,0,1,manager,Алло здравствуйте
2,0,2,client,Добрый день
3,0,3,manager,Меня зовут ангелина компания диджитал бизнес з...
4,0,4,client,Ага


In [190]:
# приведем текст к нижнему регистру
data['text'] = data['text'].apply(lambda x: x.lower())

In [191]:
# создадим списки с различными вариантами приветствия, прощания, а также шаблоны для поиска фраз,
# где менеджер представляет себя и для поиска названия компании
greetings = ['здравствуйте', 'добрый день', 'добрый вечер', 'доброе утро',
             'приветствую', 'доброго дня', 'доброго утра', 'доброго вечера']

patterns = ['(меня зовут) (\w{2,})', '(меня) (\w{2,}) (зовут)', '(мое имя) (\w{2,})',
            '(это) (\w{2,})', '(\bя) (\w{2,})']

goodbye = ['до свидания', 'всего хорошего', 'всего доброго', 'хорошего вечера']

pattern_company = ['компани. \w{2,} бизнес', 'компани. \w{2,}']

In [192]:
# функция для парсинга диалогов
def parse_dlg(dlg_id):  # на вход принимает id диалога
    df = data[(data.dlg_id == dlg_id) & (data.role == 'manager')]
    # словарь, куда мы будем складывать информацию о диалоге
    info_dlg = {'greetings': None, 'introduce': None, 'name': None, 'company_name': None,
                'goodbye': None, 'say_hello_goodbye': False}

    '''пробежимся по первым строкам - очевидно, что менеджер должен приветствовать клиента и
    представляться в одной из первых реплик'''
    for line in df['text'][:2]:
        for greeting in greetings:
            result = re.search(greeting, line)
            if result:
                # сохраняем реплику, в которой менеджер здоровается
                info_dlg['greetings'] = line
                break

        for pattern in patterns:
            res = re.search(pattern, line)
            if res:
                # сохраняем реплику, в которой менеджер представляется
                info_dlg['introduce'] = line
                info_dlg['name'] = res.group(2).title()  # сохраняем имя менеджера
                break

        for pattern in pattern_company:
            company = re.search(pattern, line)
            if company:
                company_name = company.group()
                info_dlg['company_name'] = re.sub(
                    'компани.', '', company_name).title().strip()  # сохраняем название компании
                break

    for word in goodbye:
        bye = re.search(word, df['text'].iloc[-1])
        if bye:
            # сохраняем реплику, в которой менеджер прощается
            info_dlg['goodbye'] = df['text'].iloc[-1]
            break
    # ставим флаг True, если менеджер и поздоровался,и попрощался с клиентом
    if info_dlg['greetings'] and info_dlg['goodbye']:
        info_dlg['say_hello_goodbye'] = True

    return info_dlg

In [194]:
for i in range(max(data['dlg_id'])+1):
    print(parse_dlg(i))

{'greetings': 'алло здравствуйте', 'introduce': 'меня зовут ангелина компания диджитал бизнес звоним вам по поводу продления лицензии а мы с серым у вас скоро срок заканчивается', 'name': 'Ангелина', 'company_name': 'Диджитал Бизнес', 'goodbye': 'всего хорошего до свидания', 'say_hello_goodbye': True}
{'greetings': 'алло здравствуйте', 'introduce': 'меня зовут ангелина компания диджитал бизнес звоню вам по поводу продления а мы сели обратила внимание что у вас срок заканчивается', 'name': 'Ангелина', 'company_name': 'Диджитал Бизнес', 'goodbye': 'до свидания', 'say_hello_goodbye': True}
{'greetings': 'алло здравствуйте', 'introduce': 'меня зовут ангелина компания диджитал бизнес звоню вам по поводу продления лицензии а мастера мы с вами сотрудничали по видео там', 'name': 'Ангелина', 'company_name': 'Диджитал Бизнес', 'goodbye': None, 'say_hello_goodbye': False}
{'greetings': 'алло дмитрий добрый день', 'introduce': 'добрый меня максим зовут компания китобизнес удобно говорить', 'name'